In [1]:
from decouple import config


QDRANT_URL = config('QDRANT_URL')
QDRANT_API_KEY = config('QDRANT_API_KEY')


In [ ]:
# https://qdrant.tech/documentation/quickstart/

In [2]:
from qdrant_client import QdrantClient


qdrant_client = QdrantClient(
    url=QDRANT_URL, 
    api_key=QDRANT_API_KEY,
)

print(qdrant_client.get_collections())

collections=[]


In [4]:
from fastembed import TextEmbedding


documents = [
    'FastEmbed is lighter than Transformers & Sentence-Transformers.',
    'FastEmbed is supported by and maintained by Qdrant.',
]

In [5]:
embedding_model = TextEmbedding(model_name='BAAI/bge-small-en-v1.5')

Fetching 5 files: 100%|██████████| 5/5 [00:06<00:00,  1.32s/it]


In [8]:
# https://qdrant.tech/documentation/fastembed/fastembed-quickstart/

embeddings_generator = embedding_model.embed(documents)
embeddings = list(embeddings_generator)
len(embeddings[0])  

384

In [ ]:
from qdrant_client.models import Distance, VectorParams


COLLECTION_NAME = ...

qdrant_client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

In [ ]:
from qdrant_client.models import PointStruct
from uuid import uuid4

operation_info = qdrant_client.upsert(
    collection_name=COLLECTION_NAME,
    wait=True,
    points=[
        PointStruct(
            id=uuid4(), 
            vector=embedding.tolist(), 
            payload={'text': document}
        )
        for document, embedding in zip(documents, embeddings)
    ],
)

print(operation_info)

In [ ]:
from qdrant_client.models import Filter, FieldCondition, MatchValue


search_result = qdrant_client.query_points(
    collection_name=COLLECTION_NAME,
    query=...,
    #query_filter=Filter(
    #    must=[FieldCondition(key='text', match=MatchValue(value=...))]  # metadata
    #),
    with_payload=False,
    limit=3
).points

print(search_result)